In [ ]:
from datetime import date
from os.path import basename, splitext
import re

from pyspark.sql import Row
from pyspark.sql.types import *


# TODO figure out why when scanning all records grep got a count of
# 120924 but the data frame has 120700.
# TODO make a SQL comment for dataframes for defaulted ints (like -1)


In [ ]:
retroFilePath = "retrosheet-data/*/*.EV[AN]"
# retroFilePath = "retrosheet-data/2010*/*.EV[AN]" # just the 10s

rosterPath = "retrosheet-data/*/*.ROS"
teamPath = "retrosheet-data/*/TEAM[0-9][0-9][0-9][0-9]"

DIST_PARQUET_DIR = "dist/parquet/"


In [ ]:

# the following processes the event files, ie *.EV[AN]

retrosheet = sc.newAPIHadoopFile( 
    retroFilePath, 
    'org.apache.hadoop.mapreduce.lib.input.TextInputFormat', 
    'org.apache.hadoop.io.LongWritable', 
    'org.apache.hadoop.io.Text', 
    conf={
        'textinputformat.record.delimiter':'\nid,'
    }
)


# proc to process a whole game's events. 
# by using '\nid,' as our delim above, we get called with a whole game's
# events. we resplit them on newlines so lines can be parsed with event type
# specific parsers, but first prepend the game_id and a sequence into the game's
# events to each record.
def processRecord( (k,recordsBetweenIdLines) ):

    # first, resplit the events
    events = recordsBetweenIdLines.splitlines()

    # first off, disgard non-game related records, ie they dont appear 
    # after an "id," token. examples:
    # 
    # com,"Copyright 2001 by Stats Inc."
    # com,"All Rights Reserved."
    if (events[0].startswith("com,")):
        return []
    
    # if we're not the first record, we dont still have the "id," token
    # restore the record so the id record parser can work like the others
    
    if (events[0].startswith("id,") is False):
        events[0] = "id," + events[0]
        
    # now, get the game_id that will be prepended to each record
    game_id = events[0].split(",")[1]    

    # now emit lines prepended with game_id and the seq into the game, then
    # the original lines from the file.
    return ((
        game_id,
        idx,
        record.split(",")
    ) for idx, record in enumerate(events))


# first do a flatmap to get the events between "id" records and to 
# process them using the proc above
combinedFlattenedEvents = retrosheet.flatMap(processRecord)
combinedFlattenedEvents


In [ ]:

## TODO consider refactoring game_id/seq from each schema and adding via a map
## TODO get consistent with underscore vs camelCase for columns
SCHEMA_BY_TYPE = {    
    
    # sample line: start,howar001,"Ryan Howard",0,4,3               
    # sample line: sub,waldj001,"Jordan Walden",0,0,1
    'start': { 
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 

            StructField("player_id",StringType(),False), 
            StructField("playerName",StringType(),False), 
            StructField("home",BooleanType(),False), 
            StructField("battingOrder",ShortType(),False), ## TODO can this be short
            StructField("position",ShortType(),False)
        ]), 
        'mapper': lambda rec: [
            rec[0],
            rec[1],

            rec[2][1],
            rec[2][2],
            ("0" == rec[2][3]),  ## TODO verify "0" is home
            int(rec[2][4]),
            int(re.sub('[^0-9]','',rec[2][5])) # make this a reusable proc            
        ]
    },  
    
    # sample line: play,6,1,bondb001,02,CFX,HR/9.3-H;2-H;1-H  
    'play': { 
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 

            StructField("inning",ShortType(),False), 
            StructField("topOfInning",BooleanType(),False), 
            StructField("player_id",StringType(),False), 
            StructField("count",ArrayType(ShortType(), False),True), 
            StructField("pitch_seq",StringType(),False),
            StructField("description",StringType(),False)
        ]),  
        'mapper': lambda rec: [
            rec[0],
            rec[1],

            int(rec[2][1]),             
            bool("0" == rec[2][2]),
            rec[2][3],
            [int(c) for c in list(rec[2][4])] if rec[2][4].isdigit() else [],
            rec[2][5],
            rec[2][6]            
        ]
        
    },    
    
    # sample line: com,"$Career homer 587 to pass Frank Robinson for 4th all-time"
    # or sample multi-line comment:
    # 
    # com,"$Hall caught in rundown while Winn advanced to 3B; both players"
    # com,"ended up on 3B and Winn is tagged out; Hall thought he was the one"
    # com,"who was out and stepped off the bag and is tagged out"    
    'com': { 
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 

            StructField("comment",StringType(),False), 
            StructField("key",StringType(),False), 
            StructField("value",StringType(),False)
        ]), 
        'mapper': lambda rec: [rec[0],rec[1]] + rec[2][1]
    },
    
    'info': { 
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 
            StructField("key",StringType(),False), 
            StructField("value",StringType(),False)
        ]), 
        'mapper': lambda rec: [rec[0],rec[1]] + rec[2][1:]
    },
    
    # sample line: data,er,fyhrm001,0
    # sample line: padj,harrg001,L
    # sample line: ladj,0,9
    # sample line: badj,bonib001,R
    'data': { # this is cloned below for other types
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 

            StructField("type",StringType(),False), 
            StructField("key",StringType(),False), 
            StructField("value",StringType(),False)
        ]), 
        'mapper': lambda rec: [rec[0],rec[1]] + rec[2]
    },   
    
    
    ## TODO collapse all of these com records should be one comment
    
    # sample line: com,"$Career homer 587 to pass Frank Robinson for 4th all-time"
    # or sample multi-line comment:
    # 
    # com,"$Hall caught in rundown while Winn advanced to 3B; both players"
    # com,"ended up on 3B and Winn is tagged out; Hall thought he was the one"
    # com,"who was out and stepped off the bag and is tagged out"    
    'com': { 
        'schema': StructType([    
            StructField("game_id",StringType(),False), 
            StructField("seq",ShortType(),False), 

            StructField("comment",StringType(),False)
        ]), 
        'mapper': lambda rec: [rec[0],rec[1]] + rec[2][1:]
    },
    
}

## cloned record types for similar layouts
SCHEMA_BY_TYPE['padj'] = SCHEMA_BY_TYPE['data']
SCHEMA_BY_TYPE['badj'] = SCHEMA_BY_TYPE['data']
SCHEMA_BY_TYPE['ladj'] = SCHEMA_BY_TYPE['data']

SCHEMA_BY_TYPE['sub'] = SCHEMA_BY_TYPE['start']


dfByType = {}
for recType,typeInfo in SCHEMA_BY_TYPE.iteritems():
    
    curRdd = combinedFlattenedEvents.filter(lambda rec: rec[2][0]==recType)
    if curRdd.isEmpty(): continue 
    dfByType[recType] = sqlContext.createDataFrame(curRdd.map(typeInfo['mapper']), typeInfo['schema'])
    print("%s has %d records" % (recType,dfByType[recType].count()))
    dfByType[recType].printSchema()
    dfByType[recType].write.save(DIST_PARQUET_DIR + recType, format="parquet", mode="overwrite")
    dfByType[recType].show()


print("done")


In [ ]:
# the following processes the roster files, ie *.ROS, and team files 
# TEAMyyyy. it prepends the rows in the file with its filename to extract
# things like the year

## this proc does two things, first it is a map function for a wholeTextFiles() call. It is
## meant to be called with a flatmap because it returns the rows of the file with the last part
## of the filename preprended to the records from the file. Additionally, this method can do a
## transforms on the file part which is passed in with transformPath parameter.
def getWholeTextParserWithPathForOffset(sliceFrom=None, sliceTo=None, cast=lambda x:x):
    def wholeTextParserWithPath( (path, content) ): 
        fileName = splitext(basename(path))[0]
        return [ [cast(fileName[sliceFrom:sliceTo])] + record.split(",") for record in content.splitlines()]
    return wholeTextParserWithPath


## this proc does two things, first it is a map function for a wholeTextFiles() call. It is
## meant to be called with a flatmap because it returns the rows of the file with the last part
## of the filename preprended to the records from the file. Additionally, this method can do a
## transforms on the file part which is passed in with the optional pathTransform parameter. this
## defaults to just an identity function but can be used to trim the path and cast to an int, say.
def getWholeTextParserWithPathTransform(pathTransform=lambda x:x):
    def wholeTextParserWithPath( (path, content) ): 
        fileName = splitext(basename(path))[0]
        return [ [pathTransform(fileName)] + record.split(",") for record in content.splitlines()]
    return wholeTextParserWithPath



### ROSTER files
# sample greiz001,Greinke,Zack,R,R,LAN,P
rosterSchema = StructType([
    # prepended from filename
    StructField("year",ShortType(),False), 

    # rest from record
    StructField("player_id",StringType(),False), 
    StructField("firstName",StringType(),False), 
    StructField("lastName",StringType(),False), 
    StructField("bats",StringType(),False), 
    StructField("throws",StringType(),False), 
    StructField("team",StringType(),False), 
    StructField("position",StringType(),False)
])

rosterRowsRDD = sc.wholeTextFiles(rosterPath).flatMap(
    getWholeTextParserWithPathTransform(lambda x:int(x[3:])) # trim year part and cast to int
)
rosterDataFrame = sqlContext.createDataFrame(rosterRowsRDD,rosterSchema)
rosterDataFrame.registerTempTable("roster")

teamSchema = StructType([
    # prepended from filename
    StructField("year",StringType(),False), 

    # rest from record
    StructField("team_id",StringType(),False), 
    StructField("league",StringType(),False), 
    StructField("city",StringType(),False), 
    StructField("teamName",StringType(),False)
])


### TEAM files
# sample MIL,N,Milwaukee,Brewers
teamRowsRDD = sc.wholeTextFiles(teamPath).flatMap(
    getWholeTextParserWithPathTransform(lambda x:int(x[4:]))    
)
teamDataFrame = sqlContext.createDataFrame(teamRowsRDD, teamSchema)
teamDataFrame.registerTempTable("team")

print("done")

In [ ]:

sqlContext.sql("describe roster").show()

sql = """
    select year, count(*)
    from roster
    group by year
    order by year asc

"""

sqlContext.sql(sql).show(50)

In [ ]:
for t in sqlContext.tableNames():
    print(t)
    sqlContext.sql("DESCRIBE %s" % t).show()


In [ ]:
sql = """

    select player_id, firstName, lastName, min(year), max(year), 
            count(distinct team) as numTeams, collect_set(team) as teams
    from roster
    group by player_id, firstName, lastName
    order by numTeams desc
    
"""


sqlContext.sql(sql).show(50)


In [ ]:

sql = """
    select s.player_id, s.game_id 
    from play s 
        join play d on (
            s.player_id = d.player_id 
            and s.game_id = d.game_id
            and s.description like "S%"
            and d.description like "D%"
            )

        join play t on (
            s.player_id = t.player_id 
            and s.game_id = t.game_id
            and t.description like "T%"
            )

        join play hr on (
            s.player_id = hr.player_id 
            and s.game_id = hr.game_id
            and t.description like "HR%"
            )

"""


sqlContext.sql(sql).count()
